In [1]:
import numpy as np
from typing import List

DEFAULT_MORPHOLOGY = np.array(
    [
        [0.6405, 0.3992, 0.2436, 0.0031, 0.0016, 0.4138, -0.1902],
        [0.4387, 0.2822, 0.2187, -0.0033, 0.9732, 0.2296, -0.0940],
        [0.5536, 0.3408, 0.2169, -0.0026, 0.9941, -0.0107, -0.0486],
        [0.5416, 0.3476, 0.2187, -0.0478, 0.9216, -0.2068, -0.0588],
        [0.4298, 0.2986, 0.2228, -0.1193, 0.7854, -0.3895, -0.1599],
    ],
    dtype=np.float32,
)


def rot(
    v: np.ndarray,
    p: np.ndarray,
    alpha: np.ndarray,
    beta: np.ndarray,
):
    """
    Rotation function.

    v, p: (3,) arrays representing vector and the first axis of rotation
    alpha, beta: floats representing angles to use for rotation
    """
    v = v / np.linalg.norm(v)
    p = p / np.linalg.norm(p)

    sinA = np.sin(alpha)
    cosA = np.cos(alpha)
    sinB = np.sin(beta)
    cosB = np.cos(beta)

    q = np.cross(v, p)

    p_hat = p * cosB - v * sinB
    v_hat = q * sinA + v * cosA * cosB + p * cosA * sinB

    return v_hat, p_hat


P = np.array([0, 0, 1])


def hand_landmarks_by_angles(
    angles: np.ndarray,
    morphology: np.ndarray = DEFAULT_MORPHOLOGY,
):
    """
    angles: (20,) array
    morphology: (5, 7) array - not batched
    Returns: (20, 3) array
    """
    landmarks = np.zeros((21, 3), dtype=angles.dtype)
    angles = angles.reshape(5, 4)

    # Write morphological const points
    for i in range(5):
        landmarks[4 * i + 1, :] = morphology[i, 3:6]

    for finger_index, (morph, local_angles) in enumerate(zip(morphology, angles)):
        base_idx = finger_index * 4 + 1
        bone_lengths = morph[0:3]
        joint = morph[3:6].copy()
        gamma = morph[6]

        a0, b0, a2, a3 = local_angles
        chain_angles = np.array([[a0, b0], [a2, 0], [a3, 0]])

        v = joint / np.linalg.norm(joint)

        # Get p perpendicular to v
        p_proj = v * np.dot(v, P)
        p = P - p_proj
        p = p / np.linalg.norm(p)

        # Rotate p around v according to parameter gamma
        sinG, cosG = np.sin(gamma), np.cos(gamma)
        p = p * cosG + np.cross(p, v) * sinG

        # Iterate over the chain and write the 3d points
        for j in range(3):
            l = bone_lengths[j]
            alpha, beta = chain_angles[j, 0], chain_angles[j, 1]
            v, p = rot(v, p, alpha, beta)
            joint = joint + v * l
            landmarks[base_idx + j + 1, :] = joint

    landmarks = np.concatenate(
        [landmarks[:1, :], landmarks[2:, :]], axis=0
    )  # cut thumb base

    return [l for l in landmarks]  # unroll landmarks


E = 1e-7


def irot_full_merged(
    v: np.ndarray,
    p: np.ndarray,
    v_hat: np.ndarray,
    v_hat_hat: np.ndarray,
    eps=0.7,
    eps_fallback=0.01,
):
    """
    Inverse rotation function with improved fallback.

    v, p, v_hat, v_hat_hat: (3,) arrays
    Returns: alpha, beta, v_hat_update, p_hat
    """
    v = v / np.linalg.norm(v)
    p = p / np.linalg.norm(p)
    q = np.cross(v, p)

    # Attempt primary p_hat from cross product of v_hat and v_hat_hat
    p_hat_candidate = np.cross(v_hat, v_hat_hat)
    low_norm_mask = np.linalg.norm(p_hat_candidate) < eps

    # Fallback p_hat using irot_full1 logic
    dot_v = np.dot(v_hat, v)
    dot_p = np.dot(v_hat, p)
    sinA = np.dot(v_hat, q)
    sinA = np.clip(sinA, -1 + E, 1 - E)

    cosA = np.sqrt(1 - sinA**2)
    beta_fallback = (
        np.arctan2(dot_p / cosA, dot_v / cosA) if cosA >= eps_fallback else 0
    )
    p_hat_fallback = p * np.cos(beta_fallback) - v * np.sin(beta_fallback)

    # Combine: use fallback p_hat only where primary one is invalid
    p_hat = (
        p_hat_fallback
        if low_norm_mask
        else p_hat_candidate / np.linalg.norm(p_hat_candidate)
    )

    # Continue as in irot_full
    cosB = np.dot(p_hat, p)
    sinB = -np.dot(p_hat, v)

    v_hat_update = q * sinA + v * (cosA * cosB) + p * (cosA * sinB)

    beta = np.arctan2(sinB, cosB)
    alpha = np.arcsin(sinA)

    return alpha, beta, v_hat_update, p_hat


def irot_alpha(v: np.ndarray, p: np.ndarray, v_hat: np.ndarray):
    """
    Inverse rotation function.

    v, p, v_hat: (3,) arrays
    Returns: alpha, beta, v_hat, p_hat
    """
    v = v / np.linalg.norm(v)
    p = p / np.linalg.norm(p)
    q = np.cross(v, p)

    cosA = np.dot(v_hat, v)
    cosA = np.clip(cosA, -1 + E, 1 - E)
    alpha = np.sign(np.dot(v_hat, q)) * np.arccos(cosA)
    beta = 0
    p_hat = p

    return alpha, beta, v_hat, p_hat


def inverse_hand_angles_by_landmarks(
    landmarks: List[np.ndarray],
    morphology: np.ndarray = DEFAULT_MORPHOLOGY,
):
    """
    landmarks: List[np.ndarray] - known hand landmarks in 3D space
    morphology: (5, 7) - hand morphology (not batched)

    Returns:
    angles: (20,) - recovered joint angles
    """
    angles = np.zeros((5, 4), dtype=landmarks[0].dtype)

    # NOTE: assuming 0, 4, 8, 12, 16 landmarks are existing in the morphology
    # 20 landmarks -> 21 landmarks, restoring thumb base from the morphology
    # so that now assuming 0, 1, 5, 9, 13, 17 are existing in the morphology
    thumb_base = morphology[0][3:6]

    # Insert thumb_base at the correct position
    landmarks.insert(1, thumb_base)

    for i, morph in enumerate(morphology):
        base_idx = i * 4 + 1
        joint = landmarks[base_idx] - landmarks[0]
        gamma = morph[6]

        v = joint / np.linalg.norm(joint)

        # Get p perpendicular to v
        p_proj = v * np.dot(v, P)
        p = P - p_proj
        p = p / np.linalg.norm(p)

        # Rotate p around v according to parameter gamma
        sinG, cosG = np.sin(gamma), np.cos(gamma)
        p = p * cosG + np.cross(p, v) * sinG

        target = (landmarks[base_idx + 1] - landmarks[base_idx]) / np.linalg.norm(
            landmarks[base_idx + 1] - landmarks[base_idx]
        )
        target2 = (landmarks[base_idx + 2] - landmarks[base_idx + 1]) / np.linalg.norm(
            landmarks[base_idx + 2] - landmarks[base_idx + 1]
        )
        alpha, beta, v, p = irot_full_merged(v, p, target, target2)
        angles[i, 0] = alpha
        angles[i, 1] = beta

        for j in range(1, 3):
            target = (
                landmarks[base_idx + j + 1] - landmarks[base_idx + j]
            ) / np.linalg.norm(landmarks[base_idx + j + 1] - landmarks[base_idx + j])
            alpha, beta, v, p = irot_alpha(v, p, target)
            angles[i, j + 1] = alpha

    return angles.flatten()

In [2]:
from typing import List
import numpy as np


def read_hands(file: str):
    poses: List[np.ndarray] = []
    pose_size = 20 * 3  # number of floats
    with open(file, "rb") as f:
        data = f.read()
        total_floats = len(data) // 4  # float32 = 4 bytes
        if total_floats % pose_size != 0:
            raise ValueError("File size is not a multiple of single pose size")
        array = np.frombuffer(data, dtype=np.float32)
        for i in range(0, total_floats, pose_size):
            pose = array[i : i + pose_size].reshape((20, 3))
            poses.append(pose)
    return np.stack(poses, axis=0)

In [3]:
import plotly.graph_objects as go
import numpy as np
import torch


def plot_3d_hands(
    landmarks_sequence: torch.Tensor | np.ndarray,
    ps: torch.Tensor | np.ndarray | None = None,
    ps2: torch.Tensor | np.ndarray | None = None,
):
    if isinstance(landmarks_sequence, torch.Tensor):
        landmarks_sequence = landmarks_sequence.detach().cpu().numpy()

    if ps is not None and isinstance(ps, torch.Tensor):
        ps = ps.detach().cpu().numpy()
    if ps2 is not None and isinstance(ps2, torch.Tensor):
        ps2 = ps2.detach().cpu().numpy()

    n_frames = landmarks_sequence.shape[0]
    n_pts = landmarks_sequence.shape[1]
    labels = [str(i) for i in range(n_pts)]

    fig = go.Figure()

    # Add traces for the first frame
    fig.add_trace(
        go.Scatter3d(
            x=landmarks_sequence[0, :, 0],
            y=landmarks_sequence[0, :, 1],
            z=landmarks_sequence[0, :, 2],
            mode="markers+text",
            marker=dict(size=4, color="blue"),
            text=labels,
            textposition="top center",
            textfont=dict(size=8, color="black"),
            name="Landmarks",
        )
    )

    connections = [
        (0, 1),
        (1, 2),
        (2, 3),  # Thumb
        (0, 4),
        (4, 5),
        (5, 6),
        (6, 7),  # Index
        (0, 8),
        (8, 9),
        (9, 10),
        (10, 11),  # Middle
        (0, 12),
        (12, 13),
        (13, 14),
        (14, 15),  # Ring
        (0, 16),
        (16, 17),
        (17, 18),
        (18, 19),  # Pinky
        (1, 4),
        (4, 8),
        (8, 12),
        (12, 16),  # Palm
    ]

    # Add traces for connections for the first frame
    for i, j in connections:
        fig.add_trace(
            go.Scatter3d(
                x=[landmarks_sequence[0, i, 0], landmarks_sequence[0, j, 0]],
                y=[landmarks_sequence[0, i, 1], landmarks_sequence[0, j, 1]],
                z=[landmarks_sequence[0, i, 2], landmarks_sequence[0, j, 2]],
                mode="lines",
                line=dict(color="black", width=2),
                showlegend=False,
            )
        )

    def add_beams(landmarks, vectors, color):
        for i in range(n_pts):
            start = landmarks[i]
            end = start + vectors[i] * 0.1
            fig.add_trace(
                go.Scatter3d(
                    x=[start[0], end[0]],
                    y=[start[1], end[1]],
                    z=[start[2], end[2]],
                    mode="lines",
                    line=dict(color=color, width=2),
                    showlegend=False,
                )
            )

    # Add ps (red) and ps2 (green) beams if provided for first frame
    if ps is not None:
        add_beams(landmarks_sequence[0], ps[0], "red")
    if ps2 is not None:
        add_beams(landmarks_sequence[0], ps2[0], "green")

    # Create frames for the animation
    frames = []
    for k in range(n_frames):
        frame_data = [
            go.Scatter3d(
                x=landmarks_sequence[k, :, 0],
                y=landmarks_sequence[k, :, 1],
                z=landmarks_sequence[k, :, 2],
                mode="markers+text",
                marker=dict(size=4, color="blue"),
                text=labels,
                textposition="top center",
                textfont=dict(size=8, color="black"),
                name="Landmarks",
            )
        ] + [
            go.Scatter3d(
                x=[landmarks_sequence[k, i, 0], landmarks_sequence[k, j, 0]],
                y=[landmarks_sequence[k, i, 1], landmarks_sequence[k, j, 1]],
                z=[landmarks_sequence[k, i, 2], landmarks_sequence[k, j, 2]],
                mode="lines",
                line=dict(color="black", width=2),
                showlegend=False,
            )
            for i, j in connections
        ]

        if ps is not None:
            for i in range(n_pts):
                start = landmarks_sequence[k, i]
                end = start + ps[k, i] * 0.1
                frame_data.append(
                    go.Scatter3d(
                        x=[start[0], end[0]],
                        y=[start[1], end[1]],
                        z=[start[2], end[2]],
                        mode="lines",
                        line=dict(color="red", width=2),
                        showlegend=False,
                    )
                )

        if ps2 is not None:
            for i in range(n_pts):
                start = landmarks_sequence[k, i]
                end = start + ps2[k, i] * 0.1
                frame_data.append(
                    go.Scatter3d(
                        x=[start[0], end[0]],
                        y=[start[1], end[1]],
                        z=[start[2], end[2]],
                        mode="lines",
                        line=dict(color="green", width=2),
                        showlegend=False,
                    )
                )

        frames.append(go.Frame(data=frame_data, name=str(k)))

    fig.frames = frames

    # Add slider and play/pause button
    sliders = [
        {
            "pad": {"b": 10, "t": 10},
            "len": 0.9,
            "x": 0.1,
            "xanchor": "left",
            "y": 0,
            "yanchor": "top",
            "steps": [
                {
                    "args": [
                        [f.name],
                        {
                            "frame": {"duration": 33.33, "redraw": True},
                            "mode": "immediate",
                            "transition": {"duration": 0},
                        },
                    ],
                    "label": str(k),
                    "method": "animate",
                }
                for k, f in enumerate(frames)
            ],
        }
    ]

    fig.update_layout(
        updatemenus=[
            {
                "buttons": [
                    {
                        "args": [
                            None,
                            {
                                "frame": {"duration": 33.33, "redraw": True},
                                "fromcurrent": True,
                                "transition": {"duration": 0},
                            },
                        ],
                        "label": "Play",
                        "method": "animate",
                    },
                    {
                        "args": [
                            [None],
                            {
                                "frame": {"duration": 0, "redraw": True},
                                "mode": "immediate",
                                "transition": {"duration": 0},
                            },
                        ],
                        "label": "Pause",
                        "method": "animate",
                    },
                ],
                "direction": "left",
                "pad": {"r": 10, "t": 30},
                "showactive": False,
                "type": "buttons",
                "x": 0.1,
                "xanchor": "right",
                "y": 0,
                "yanchor": "top",
            }
        ],
        sliders=sliders,
        scene=dict(
            xaxis=dict(
                range=[2, -2],
                title="X",
            ),
            yaxis=dict(
                range=[-1, 3],
                title="Y",
            ),
            zaxis=dict(
                range=[-2, 2],
                title="Z",
            ),
            camera=dict(eye=dict(x=0.2, y=0.2, z=0.2)),
        ),
        scene_aspectmode="cube",
        title="3D Hand Pose with Point Labels",
        margin=dict(l=0, r=0, b=0, t=30),
        height=400,
    )

    fig.show()

In [ ]:
import numpy as np
from scipy.interpolate import interp1d
import numpy as np
from scipy.interpolate import make_interp_spline
from scipy.interpolate import CubicSpline


# Perfect anchor points, a little robotic in between, but still good enough
def _resample_linear(array: np.ndarray, target_size: int) -> np.ndarray:
    """
    Resample a signal array to a target size using linear interpolation

    Args:
        array (np.ndarray): Input signal array of shape (n_samples, ...)
        target_size (int): Desired number of samples in output

    Returns:
        np.ndarray: Resampled array of shape (target_size, ...)
    """
    # Generate x coordinates for original and target
    x = np.linspace(0, 1, array.shape[0])
    x_new = np.linspace(0, 1, target_size)

    # Preserve shape of additional dimensions
    orig_shape = array.shape
    n_channels = np.prod(orig_shape[1:]) if len(orig_shape) > 1 else 1

    # Reshape to 2D array (samples, channels)
    y = array.reshape(-1, n_channels)

    # Create interpolation function
    f = interp1d(x, y, axis=0, kind="linear")

    # Interpolate
    resampled = f(x_new)

    # Reshape back to original dimensions
    if len(orig_shape) > 1:
        resampled = resampled.reshape((target_size,) + orig_shape[1:])

    return resampled


# Overhoots making imposible hand poses if the original fps is too low, however hides the noise of the initial data and not so robotic looking
def _resample_bspline(
    array: np.ndarray, target_size: int, order: int = 3
) -> np.ndarray:
    """
    Resample a signal array to a target size using B-spline interpolation

    Args:
        array (np.ndarray): Input signal array of shape (n_samples, ...)
        target_size (int): Desired number of samples in output
        order (int): Order of the B-spline

    Returns:
        np.ndarray: Resampled array of shape (target_size, ...)
    """
    # Generate x coordinates for original and target
    x = np.linspace(0, 1, array.shape[0])
    x_new = np.linspace(0, 1, target_size)

    # Preserve shape of additional dimensions
    orig_shape = array.shape
    n_channels = np.prod(orig_shape[1:]) if len(orig_shape) > 1 else 1

    # Reshape to 2D array (samples, channels)
    y = array.reshape(-1, n_channels)

    # Initialize array for resampled data
    resampled = np.zeros((target_size, n_channels))

    # Fit B-spline for each channel separately
    for ch in range(n_channels):
        spl = make_interp_spline(x, y[:, ch], k=order)
        resampled[:, ch] = spl(x_new)

    # Reshape back to original dimensions
    if len(orig_shape) > 1:
        resampled = resampled.reshape((target_size,) + orig_shape[1:])

    return resampled


# Looks like bspline, but faster
def _resample_cubic_spline(array: np.ndarray, target_size: int) -> np.ndarray:
    """
    Resample a signal array to a target size using cubic spline interpolation

    Args:
        array (np.ndarray): Input signal array of shape (n_samples, ...)
        target_size (int): Desired number of samples in output

    Returns:
        np.ndarray: Resampled array of shape (target_size, ...)
    """
    # Generate x coordinates for original and target
    x = np.linspace(0, 1, array.shape[0])
    x_new = np.linspace(0, 1, target_size)

    # Preserve shape of additional dimensions
    orig_shape = array.shape
    n_channels = np.prod(orig_shape[1:]) if len(orig_shape) > 1 else 1

    # Reshape to 2D array (samples, channels)
    y = array.reshape(-1, n_channels)

    # Create cubic spline interpolation function
    cs = CubicSpline(x, y, axis=0)

    # Interpolate
    resampled = cs(x_new)

    # Reshape back to original dimensions
    if len(orig_shape) > 1:
        resampled = resampled.reshape((target_size,) + orig_shape[1:])

    return resampled


from scipy.interpolate import Akima1DInterpolator


# Still looks robotic, but less rototic then linear, has no problem with overhooting
def _resample_akima(array: np.ndarray, target_size: int) -> np.ndarray:
    """
    Resample a signal array to a target size using Akima interpolation

    Args:
        array (np.ndarray): Input signal array of shape (n_samples, ...)
        target_size (int): Desired number of samples in output

    Returns:
        np.ndarray: Resampled array of shape (target_size, ...)
    """
    x = np.linspace(0, 1, array.shape[0])
    x_new = np.linspace(0, 1, target_size)

    orig_shape = array.shape
    n_channels = np.prod(orig_shape[1:]) if len(orig_shape) > 1 else 1

    y = array.reshape(-1, n_channels)

    resampled = np.zeros((target_size, n_channels))
    for ch in range(n_channels):
        akima = Akima1DInterpolator(x, y[:, ch])
        resampled[:, ch] = akima(x_new)

    if len(orig_shape) > 1:
        resampled = resampled.reshape((target_size,) + orig_shape[1:])

    return resampled


from scipy.interpolate import PchipInterpolator


# Looks exactly like akima, but is slower a bit
def _resample_monotone_cubic(array: np.ndarray, target_size: int) -> np.ndarray:
    """
    Resample a signal array to a target size using monotone cubic interpolation

    Args:
        array (np.ndarray): Input signal array of shape (n_samples, ...)
        target_size (int): Desired number of samples in output

    Returns:
        np.ndarray: Resampled array of shape (target_size, ...)
    """
    x = np.linspace(0, 1, array.shape[0])
    x_new = np.linspace(0, 1, target_size)

    orig_shape = array.shape
    n_channels = np.prod(orig_shape[1:]) if len(orig_shape) > 1 else 1

    y = array.reshape(-1, n_channels)

    resampled = np.zeros((target_size, n_channels))
    for ch in range(n_channels):
        pchip = PchipInterpolator(x, y[:, ch])
        resampled[:, ch] = pchip(x_new)

    if len(orig_shape) > 1:
        resampled = resampled.reshape((target_size,) + orig_shape[1:])

    return resampled

In [5]:
rec = read_hands("../dataset.rec")

In [7]:
import time


def test_resample(start, stop, resample, k, divide=True, plot=False):
    joint_samples = rec[start:stop]
    angle_samples = np.stack(
        [inverse_hand_angles_by_landmarks([l for l in hand]) for hand in joint_samples]
    )

    start_time = time.time()
    angle_samples = resample(
        angle_samples, angle_samples.shape[0] // (k if divide else 1)
    )
    angle_samples = resample(angle_samples, angle_samples.shape[0] * k)
    end_time = time.time()

    joint_samples = np.stack([hand_landmarks_by_angles(a) for a in angle_samples])
    if plot:
        plot_3d_hands(joint_samples)
    else:
        print(f"{resample} resampling time: {end_time - start_time:.4f} seconds")


testing = [
    _resample_linear,
    _resample_akima,
    _resample_monotone_cubic,
    _resample_cubic_spline,
    _resample_bspline,
]

for resample in testing:
    test_resample(0, 1000, resample, 4)

for resample in testing:
    test_resample(240, 300, resample, 10, plot=True)

<function _resample_linear at 0x000001D70893A290> resampling time: 0.0000 seconds
<function _resample_akima at 0x000001D7523F4EE0> resampling time: 0.0063 seconds
<function _resample_monotone_cubic at 0x000001D7523F4F70> resampling time: 0.0083 seconds
<function _resample_cubic_spline at 0x000001D7523F4E50> resampling time: 0.0022 seconds
<function _resample_bspline at 0x000001D732C32050> resampling time: 0.0063 seconds
